## Pair Selection using Machine Learning

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import arbitragelab as al
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = False
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.right'] = False

prices_df = pd.read_csv('../../data/data.csv').set_index('Date').dropna()
prices_df.index = pd.to_datetime(prices_df.index)

prices_df = prices_df.last('10Y')

### Dimensionality Reduction

PCA를 사용해서 각 자산의 최적 군집을 찾는다. PCA로 차원 축소를 하기 전, 가격 데이터를 수익률 데이터로 바꾼 뒤, 평균을 표준편차로 나눠서 정규화를 진행해 주어야 한다.

Returns 

$$ R_{i, t} = \frac{P_{i,t} - P_{i,t-1}} {P_{i,t-1}} $$


Data Normalization

$$ Y_{i} =  \frac{R_{i} - \bar{R_{i}}} {\sigma_{i}} $$


In [ ]:
from arbitragelab.ml_approach import OPTICSDBSCANPairsClustering

ps = OPTICSDBSCANPairsClustering(prices_df)

# Here the first parameter is the number of features to reduce to.
ps.dimensionality_reduction_by_components(5)

# The following will plot the feature vector from the previous method call.
ps.plot_pca_matrix();